# Introduction

Takes exploratory data and fits variations of the DDM.

DDM: average-signal

Details: Take every signal that the subject saw in a trial, and average their values. Feed that into a DDM as a time-invariant signal.

## Preamble

In [1]:
# Install (package verification, PyDDM, timer, parallelization)
#!pip install paranoid-scientist
#!pip install pyddm
#!pip install pytictoc  
#!pip install pathos  

In [2]:
# Libraries
import os
from pytictoc import TicToc
import csv
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import multiprocessing
import pyddm as ddm
from pyddm import Model, Sample, FitResult, Fittable, Fitted, ICPoint, set_N_cpus
from pyddm.models import NoiseConstant, BoundConstant, OverlayChain, OverlayNonDecision, OverlayUniformMixture, LossRobustBIC
from pyddm.functions import fit_adjust_model, display_model
import pyddm.plot

In [3]:
# Directories
datadir = "D:\\OneDrive - California Institute of Technology\\PhD\\Rangel Lab\\2023-common-consequence\\data\\processed_data"
ddmdir = "D:\\OneDrive - California Institute of Technology\\PhD\\Rangel Lab\\2023-common-consequence\\analysis\\outputs\\ddm"

In [4]:
# Parallel settings
ncpu = multiprocessing.cpu_count()-1 # always save one core
ncpu

11

## Import and Clean Raw Data

rawdata_in: odd trials used for in-sample data for model fitting.

rawdata_out: even trials used for out-sample data for model predictions.

In [5]:
# Import
rawdata = pd.read_csv(datadir+"\\pilotdata.csv")

# Rename variables
data = rawdata.rename(columns={
    "Participant.Private.ID":"subject", 
    "Trial.Number":"trial",
    "Reaction.Time":"rt",
    "Response":"choice",
})

# RT in s.
data.rt = data.rt/1000

# Expected value of left vs right.
data['vL'] = data.L1H*data.L1PrH + data.L1M*data.L1PrM + data.L1L*data.L1PrL
data['vR'] = data.L2H*data.L2PrH + data.L2M*data.L2PrM + data.L2L*data.L2PrL

# Expected value difference (L-R)
data['vDiff'] = np.round(data.vL - data.vR,2)


# Only keep the variables of interest
voi = ['subject','trial','rt','choice','vDiff','vL','vR']
data = data[voi]

# Ceiling of Maximum RT for PyDDM
maxRT = math.ceil(max(data.rt))

# Save & display

data

,subject,trial,rt,choice,vDiff,vL,vR
0,9156374,1,5.968,0,-2.25,30.00,32.25
1,9156374,2,3.920,1,3.75,30.00,26.25
2,9156374,3,4.045,1,5.25,30.00,24.75
3,9156374,4,4.630,1,-0.75,29.25,30.00
4,9156374,5,5.052,1,0.00,30.00,30.00
...,...,...,...,...,...,...,...
1003,9158535,36,3.026,0,3.75,12.75,9.00
1004,9158535,37,1.884,0,-0.75,8.25,9.00
1005,9158535,38,1.970,0,0.00,9.00,9.00
1006,9158535,39,4.412,1,3.75,9.00,5.25


---
# DDM: Condition Invariant

## Fit the average stimulus DDM

In [6]:
# Create a drift subclass so drift can vary with stimulus.
class DriftRate(ddm.models.Drift):
  name = "Drift depends linearly on value difference"
  required_parameters = ["driftrate"] # Parameters we want to include in the model.
  required_conditions = ["vDiff"] # The column in your sample data that modulates the parameters above.
  def get_drift(self, conditions, **kwargs):
    return self.driftrate * conditions["vDiff"]

# Define the model.
model_ci = Model(name="Standard DDM that does not distinguish between AB and A'B' choices",
                 drift=DriftRate(driftrate=Fittable(minval=-1, maxval=1.5)),
                 noise=NoiseConstant(noise=Fittable(minval=.001, maxval=2)),
                 bound=BoundConstant(B=1),
                 IC=ICPoint(x0=Fittable(minval=-.99, maxval=.99)),
                 overlay=OverlayNonDecision(nondectime=Fittable(minval=0, maxval=.1)),
                 dx=.01, dt=.01, T_dur=maxRT,  # dx: spatial grid for evidence space (-B to B, in dx bins), dt: time step in s. See Shin et al 2022 Fig 4 for why I set dx=dt.
                 choice_names=("Left","Right"))

In [7]:
# Interactive plot! Play with the variables!
vDiff = np.sort(data.vDiff.unique())
pyddm.plot.model_gui_jupyter(model=model_ci, conditions={"vDiff":vDiff.tolist()})

Output()

In [ ]:
# Only run this if specified at the start. Otherwise, just load pre-saved weights.
print("DDM: Condition Invariant.")

# Iterate through subjects.
subnums = np.sort(data.subject.unique())
for subnum in subnums:

    # Progress tracker.
    print("============================================================================")
    print("Subject " + str(subnum))

    # Subset the data.
    subdata = data[data["subject"]==subnum]

    # Create a sample object from our data. Sample objects are the standard input for pyDDM fitting functions.
    ddm_data = Sample.from_pandas_dataframe(subdata, rt_column_name="rt", choice_column_name="choice", choice_names=("Left","Right"))

    # Fit the model and show it off. Keep track of how long it took to estimate the parameters.
    clock = TicToc() # Timer
    clock.tic()
    set_N_cpus(ncpu) # Parallelize
    fit_model_ci = fit_adjust_model(sample=ddm_data, model=model_ci,
                                    fitting_method="differential_evolution",
                                    lossfunction=LossRobustBIC,
                                    verbose=False)
    clock.toc("Fitting subject " + str(subnum) + " took")
    display_model(fit_model_ci)

    # Save
    filename = ddmdir + "fit_model_ci_" + str(subnum) + ".txt"
    with open(filename, "w") as f:
      f.write(repr(fit_model_ci))

DDM: Condition Invariant.
Subject 9156374


## Extract parameters and negative log likelihood for the model objects

In [ ]:
# Placeholders
model_as_bic = []
model_as_drift = []
model_as_noise = []
model_as_bias = []
model_as_ndt = []

# Iterate through subjects.
subnums = np.sort(data.subject.unique())
for subnum in subnums:
    
    # Load
    filename = tempdir + "fit_model_as_" + str(subnum) + ".txt"
    with open(filename, "r") as f:
        model_loaded = eval(f.read())

    # Negative Log Likelihood.
    model_as_bic.append(model_loaded.get_fit_result().value())
    
    # Fitted parameters.
    model_as_drift.append(model_loaded.parameters()['drift']['driftstim'])
    model_as_noise.append(model_loaded.parameters()['noise']['noise'])
    model_as_bias.append(model_loaded.parameters()['IC']['x0'])
    model_as_ndt.append(model_loaded.parameters()['overlay']['nondectime'])
    
d = {'bic':model_as_bic, "drift":model_as_drift, "noise":model_as_noise, "bias":model_as_bias, "ndt":model_as_ndt}
indiv_model_as = pd.DataFrame(data=d)
indiv_model_as

## Means of BIC and Estimates

Confidence intervals assume normal distribution.

In [ ]:
summstats_model_as = pd.DataFrame(data={"mean":indiv_model_as.mean(), 
                                        "se":indiv_model_as.sem(),
                                        "ci_lower":indiv_model_as.mean()-1.96*indiv_model_as.sem(),
                                        "ci_upper":indiv_model_as.mean()+1.96*indiv_model_as.sem()}).T
summstats_model_as